In [1]:
import requests
from datetime import datetime
import io
import pandas as pd

In [2]:
# api-endpoint
sensor_id = 90651 #sensor-index
sensor_name = 'Orkhon_01'
year = 2022

URL = f"https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv"
  
XAPI = '' #API from the orgnization

start_time = '01-01-2022'
end_time = '12-31-2022'
average = '1440' #mins

fields = ','.join(['pm2.5_atm','pm2.5_cf_1']) #returns average for channel A and B but excluding downgraded channels and using CF=1 variant for indoor, ATM variant for outdoor devices.
begindate = datetime.strptime(start_time, '%m-%d-%Y')
enddate   = datetime.strptime(end_time, '%m-%d-%Y')

begindate = datetime.timestamp(begindate)
enddate = datetime.timestamp(enddate)

# defining a params dict for the parameters to be sent to the API
PARAMS = {'start_timestamp':begindate, 'end_timestamp':enddate, 'average':average, 'fields':fields}
HEADER = {'X-API-Key':XAPI}  
# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS, headers = HEADER) 
# extracting data
rawData = pd.read_csv(io.StringIO(r.content.decode('utf-8')))

In [3]:
def tstodt(ts):
    return datetime.fromtimestamp(ts)

In [4]:
rawData['time_stamp'] = rawData['time_stamp'].apply(tstodt)
rawData = rawData.sort_values(by=['time_stamp'])
rawData = rawData.reset_index(drop=True)
rawData['time_stamp'] = rawData['time_stamp'].dt.date

In [7]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time_stamp    362 non-null    object 
 1   sensor_index  362 non-null    int64  
 2   pm2.5_atm     362 non-null    float64
 3   pm2.5_cf_1    362 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 11.4+ KB


In [9]:
rawData.to_csv(f'./{sensor_name}_{year}.csv')